#### Sideloading data

In this notebook we'll learn how to add data from sources other than the automated ones into pyWaPOR. We start by creating NDVI, Albedo and LST maps from a manually downloaded Landsat tile and then continue to incorporate that data into `pre_et_look`.

##### Landsat

First we install pywapor, in case it's not installed yet.

In [ ]:
!pip install pywapor==2.3.4 --quiet

Then we'll define the usual parameters.

In [1]:
import pywapor

project_folder = r"/Users/hmcoerver/pywapor_notebooks"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]
startdate = "2021-07-01"
enddate = "2021-07-11"

The function that transforms Landsat Collection-2 Level-2 scenes into `ndvi`, `r0` and `lst` maps, takes one mandatory input variable, which describes in which folder it should look for `tar`-files containing Landsat scenes. So let's define that variable.

In [ ]:
landsat_folder = r"/Users/hmcoerver/pywapor_notebooks/my_landsat_folder"

You can see that for now there is only 1 file in this folder.

In [ ]:
%ls $landsat_folder

Optionally, you can pass a bounding-box parameter to `pywapor.collect.Landsat.C2L2SP.main`, which will reproject the data to `epsg:4326` and clip the scene to our AOI. We'll do that here, because it will reduce computation time and because large parts of the scene contain desert, which is not very interesting anyway.

Then we run the function.

In [ ]:
all_files = pywapor.collect.Landsat.C2L2SP.main(landsat_folder, bb = (latlim, lonlim))

the output, `all_files`, contains three lists, with the paths to any tif-files that have been created.

In [ ]:
all_files

Let's have a quick look at them before we continue.

In [ ]:
import matplotlib.pyplot as plt

ndvi_data = pywapor.general.processing_functions.open_as_array(all_files[0][0])
r0_data = pywapor.general.processing_functions.open_as_array(all_files[1][0])
lst_data = pywapor.general.processing_functions.open_as_array(all_files[2][0])

fig = plt.figure(1)
fig.clf()
fig.set_size_inches(9.69, 5.27)
axs = fig.subplots(1, 3, sharex=False, sharey=False).flatten()

pywapor.post_et_look.plot_img(axs[0], ndvi_data, "ndvi", "NDVI [-]")
pywapor.post_et_look.plot_img(axs[1], r0_data, "r0", "Albedo [-]")
pywapor.post_et_look.plot_img(axs[2], lst_data, "lst", "LST [K]")

That doesn't look good, does it? Landsat 7 has some scanline problems, as a result of which many scenes have stripes of missing data. Because we are zoomed out quite far in these plots, it looks as if there is no data at all. Lets zoom in a little.

In [ ]:
fig = plt.figure(2)
fig.clf()
fig.set_size_inches(9.69, 4.77)
axs = fig.subplots(1, 3, sharex=False, sharey=False).flatten()

pywapor.post_et_look.plot_img(axs[0], ndvi_data[600:1200, 1300:1900], "ndvi", "NDVI [-]")
pywapor.post_et_look.plot_img(axs[1], r0_data[600:1200, 1300:1900], "r0", "Albedo [-]")
pywapor.post_et_look.plot_img(axs[2], lst_data[600:1200, 1300:1900], "lst", "LST [K]")

Now you can see that there actually is data. 

Also notice the circle of missing data in the lst-map. Each level-2 Landsat scene comes with a map indicating the uncertainty in the given lst-values. By default, `pywapor.collect.Landsat.C2L2SP.main` automatically masks out any `lst` values that have an uncertainty greater than 2.5 K. You can adjust this value by passing the keyword argument `max_lst_uncertainty` to the function. For example like this: `all_files = pywapor.collect.Landsat.C2L2SP.main(landsat_folder, max_lst_uncertainty = 5.0)`.

We can check our Landsat folder again.

In [ ]:
%ls $landsat_folder

As you can see, there are now three folders next to our tar-file, which contain the maps that have been created.

##### Sideloading

Now that we have some data that we would like to include in `pre_et_look`, we can define a custom level like we've done in the previous notebook.

In [2]:
my_custom_level = {
        # Main inputs
        "ndvi":         ["LS7"],
        "r0":           ["LS7"],
        "lst":          ["LS7"],
        "lulc":         ["WAPOR"],
        "z":            ["SRTM"],
        "p_24":         ["CHIRPS"],
        "ra_24":        ["MERRA2"],

        # Daily meteo 
        't_air_24':     ["GEOS5"],
        't_air_min_24': ["GEOS5"], 
        't_air_max_24': ["GEOS5"],
        'u2m_24':       ["GEOS5"],
        'v2m_24':       ["GEOS5"],
        'p_air_0_24':   ["GEOS5"],
        'qv_24':        ["GEOS5"],

        # Instanteneous meteo
        "t_air_i":      ["GEOS5"],
        "u2m_i":        ["GEOS5"],
        "v2m_i":        ["GEOS5"],
        "qv_i":         ["GEOS5"],
        "wv_i":         ["GEOS5"],
        "p_air_i":      ["GEOS5"],
        "p_air_0_i":    ["GEOS5"],

        # Temporal constants
        "lw_offset":    ["STATICS"],
        "lw_slope":     ["STATICS"],
        "r0_bare":      ["STATICS"],
        "r0_full":      ["STATICS"],
        "rn_offset":    ["STATICS"],
        "rn_slope":     ["STATICS"],
        "t_amp_year":   ["STATICS"],
        "t_opt":        ["STATICS"],
        "vpd_slope":    ["STATICS"],
        "z_oro":        ["STATICS"],

        # Level name
        "level_name": "sideloading_level",
}

What's different from the previous notebook however, is that now we also need to tell `pre_et_look` where to find these new sources (i.e. `"LS7"`). We do that with another dictionary.

In [3]:
extra_source_locations = {
    ("LS7", "ndvi"): r"/Users/hmcoerver/pywapor_notebooks/my_landsat_folder/ndvi",
    ("LS7", "r0"): r"/Users/hmcoerver/pywapor_notebooks/my_landsat_folder/r0",
    ("LS7", "lst"): r"/Users/hmcoerver/pywapor_notebooks/my_landsat_folder/lst",
}

For each entry of the form "(`source`, `parameter`): `folder_path`", `pre_et_look` will look for files in the folder given by `folder_path` which look like this: `{parameter}_{source}_*_%Y.%m.%d.%H.%M.tif`.

We'll also pass some diagnostic points to `pre_et_look`, so that the function will create some graphs at several POIs  (as we've also done in the `composites` notebook).

In [4]:
diagnostics = { # label          # lat      # lon
                "water":	    (29.44977,	30.58215),
                "desert":	    (29.12343,	30.51222),
                "agriculture":	(29.32301,	30.77599),
                "urban":	    (29.30962,	30.84109),
                }

And since we're not adjusting the `composite_length` argument, the function will use the default value of `"DEKAD"`.

In [5]:
ds_in, fh_in = pywapor.pre_et_look.main(project_folder, startdate, enddate, latlim, lonlim, level = my_custom_level, 
    diagnostics = diagnostics, extra_source_locations = extra_source_locations)


> PRE_ET_LOOK
    # ndvi
    --> Collected 1 LS7 file(s).
    --> Resampling resolution is ~30 meter.
    --> Resampling datasets.
    --> Calculating composites.
    --> Calculating diagnostics.
    # p_24
    --> Downloading CHIRPS.
Tile: 11 / 11: 0.00Bytes [00:00, ?Bytes/s]
    --> Resampling datasets.
    --> Calculating composites.
    --> Calculating diagnostics.
    # se_root
    > PRE_SE_ROOT
        # ndvi
        --> Collected 1 LS7 file(s).
        --> Resampling resolution is ~30 meter.
        --> Resampling datasets.
        # lst
        --> Collected 1 LS7 file(s).
        --> Resampling datasets.
        > METEO
            # t_air_i
            --> Downloading GEOS5 (3-hourly), t2m.
Tile: 1 / 1: 0.00Bytes [00:00, ?Bytes/s]
            --> Applying 'kelvin_to_celsius' to `t_air_i` from GEOS5.
            # u2m_i
            --> Downloading GEOS5 (3-hourly), u2m.
Tile: 1 / 1: 0.00Bytes [00:00, ?Bytes/s]
            # v2m_i
            --> Downloading GEOS5 (3-hourly), v